In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T

In [4]:
VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True,
                           transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)

Files already downloaded and verified


In [21]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)

Conv2d       (192, 29, 29)  (4, 4)   (1, 1)  
AvgPool2d    (192, 15, 15)  (3, 3)   (2, 2)  
MaxPool2d    (192, 5, 5)    (3, 3)   (3, 3)  
LeakyReLU    (192, 5, 5)    (0.1)   
Conv2d       (16, 2, 2)     (1, 1)   (3, 3)  
AvgPool2d    (16, 2, 2)     (4, 4)   (2, 2)  
MaxPool2d    (16, 2, 2)     (3, 3)   (1, 1)  
LeakyReLU    (16, 2, 2)     (0.001) 
Conv2d       (256, 1, 1)    (3, 3)   (4, 4)  
BatchNorm2d  (256, 1, 1)   
ReLU         (256, 1, 1)   
LeakyReLU    (256, 1, 1)    (0.001) 
GlobalAvgPool2d (256, 1, 1)   
Flatten      256           
Linear       10            



In [22]:
model_tmpl.layers[0].templates[0]

Conv2d       (192, 29, 29)  (4, 4)   (1, 1)  

In [23]:
fixed_model_gpu

Sequential(
  (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(1, 1))
  (1): AvgPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (3): LeakyReLU(negative_slope=0.1)
  (4): Conv2d(192, 16, kernel_size=(1, 1), stride=(3, 3))
  (5): AvgPool2d(kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
  (6): MaxPool2d(kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
  (7): LeakyReLU(negative_slope=0.001)
  (8): Conv2d(16, 256, kernel_size=(3, 3), stride=(4, 4), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): LeakyReLU(negative_slope=0.001)
  (12): AvgPool2d(kernel_size=(1, 1), stride=(1, 1), padding=0)
  (13): Flatten()
  (14): Linear(in_features=256, out_features=10, bias=True)
)

In [24]:
loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

# express train
t, val_acc, train_acc = train_express_gpu(model = fixed_model_gpu,
                                          train = True,
                                          loader = loader_gpu,
                                          milestones = [],
                                          num_epoch = 1,
                                          verbose = True)

Finished in 30.5s 
val_acc: 0.224, train_acc: 0.185


In [8]:
print(len(cifar10))

50000
